# Fine-tuning the YOLO model from Recorded Images

# Requirements

In [ ]:
!pip install roboflow
!pip install ultralytics

# Download dataset from Roboflow

In [ ]:
import os
import pathlib
import shutil
import torch
from ultralytics import YOLO
from roboflow import Roboflow
from dotenv import load_dotenv

load_dotenv()

current_dir = pathlib.Path.cwd()
ROOT_DIR = current_dir  # For Colab
#ROOT_DIR = current_dir.parent # For local dev

DATASET_DIR = ROOT_DIR / "data" / "roboflow_chess_dataset"
EPOCHS = 100


def download_dataset():
    ROBOFLOW_API_KEY = os.getenv("ROBOFLOW_API_KEY")

    rf = Roboflow(api_key=ROBOFLOW_API_KEY)
    project = rf.workspace("test-wzcdu").project("chess-pieces-detection-camera-pxzh6")
    version = project.version(2)
    dataset = version.download("yolov8")

    print("Original Dataset location: ", dataset.location)
    print("Moving dataset to: ", DATASET_DIR)
    shutil.move(dataset.location, str(DATASET_DIR))


def train_yolo_model():

    # Check if GPU/MPS is available
    if torch.cuda.is_available():
        device = "cuda"
    elif torch.backends.mps.is_available():
        device = "mps"
    else:
        device = "cpu"

    # Load a pre-trained YOLO model (you can choose different sizes: yolov8n, yolov8s, yolov8m, yolov8l, yolov8x)
    model = YOLO("yolov8s.pt")

    print("Using device: ", device)
    print("Dataset file: ", os.path.join(str(DATASET_DIR), "data.yaml"))
    print("Project: ", str(ROOT_DIR / "models" / "yolo" / "finetuned"))

    # Train the model on the custom dataset
    model.train(
        data=os.path.join(str(DATASET_DIR), "data.yaml"),
        epochs=EPOCHS,
        imgsz=1216,  # Multiple of 32 (1216)
        batch=3,
        device=device,
        project=str(ROOT_DIR / "models" / "yolo" / "finetuned"),
        name="chess_yolov8m_finetuned",
        exist_ok=True,
        # augment=False,
        # plots=False,
        # cache=False,
        # workers=0,
        # val=False,  # Skip validation initially
    )

    # Evaluate the model
    results = model.val()
    print(results)


def train_with_cli():
    import subprocess

    cmd = [
        "yolo",
        "train",
        f"data={DATASET_DIR}/data.yaml",
        "model=yolov8s.pt",
        "epochs=1",
        "imgsz=640",
        "batch=1",
        f"project={ROOT_DIR}/models/yolo/finetuned",
        "name=chess_cli_test",
        "exist_ok=True",
    ]

    result = subprocess.run(cmd, capture_output=True, text=True)
    print("STDOUT:", result.stdout)
    print("STDERR:", result.stderr)
    return result.returncode == 0

In [ ]:
download_dataset()    

In [ ]:
train_yolo_model()

# Zip to download

In [ ]:
!zip -r models.zip models/

# Fine-tuning cli

In [ ]:
!yolo task=detect mode=train model=yolov8m data={dataset.location}/data.yaml epochs=100 imgsz=1000